In [1]:
from Dronesimscape import Dronesimscape
from simulink_gym import logger

logger.setLevel(10)


In [2]:
# Create training environment:
env = Dronesimscape(stop_time=10, step_size=0.001, timestep=0.001, model_debug=False)


INFO: Starting Matlab engine
INFO: Adding components to Matlab path
INFO: Creating simulation input object for model Dronesimscape.slx


In [3]:
# Reset environment:
state = env.reset()


INFO: recv_socket: Socket not connected, nothing to close
INFO: send_socket: Socket not connected, nothing to close


In [ ]:
# Execute single step in the environment:
# action = env.action_space.sample()
# state, reward, done, info = env.step(action)


In [ ]:
# action = env.action_space.sample()
# state

In [4]:
from agent.PPOAgent import *
total_test_episodes = 10    # total num of testing episodes

K_epochs = 80               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003           # learning rate for actor
lr_critic = 0.001           # learning rate for critic
env_name = 'Dronesimscape'
directory = "agent/PPO_preTrained/" + env_name + '/'
checkpoint_path = directory + "PPO_{}.pth".format(env_name)

Device set to : NVIDIA GeForce RTX 3060 Laptop GPU


In [6]:

print("============================================================================================")


################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "Dronesimscape"
has_continuous_action_space = True

max_ep_len = 1000              # max timesteps in one episode
max_training_timesteps = int(1e8)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None


#####################################################


## Note : print/log frequencies should be > than max_ep_len


################ PPO hyperparameters ################


update_timestep = max_ep_len * 4      # update policy every n timesteps
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)

print("training environment name : " + env_name)
#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


################### checkpointing ###################

run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

directory = "agent/PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)


checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, random_seed, run_num_pretrained)
print("save checkpoint path : " + checkpoint_path)

#####################################################


############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)

print("model saving frequency : " + str(save_model_freq) + " timesteps")
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("--------------------------------------------------------------------------------------------")

if has_continuous_action_space:
    print("Initializing a continuous action space policy")
    print("--------------------------------------------------------------------------------------------")
    print("starting std of action distribution : ", action_std)

else:
    print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps") 
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim=19, action_dim=10, lr_actor = lr_actor, lr_critic = lr_critic, gamma = gamma, K_epochs = K_epochs, eps_clip = eps_clip, has_continuous_action_space = True, action_std_init=0.1)
ppo_agent.load(checkpoint_path)

# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")

# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0
action_std_decay_freq = int(1e5)
action_std_decay_rate = 1e-3
min_action_std = 1e-3

# training loop
while time_step <= max_training_timesteps:
    
    state = env.reset()
    current_ep_reward = 0

    for t in range(1, max_ep_len+1):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state, reward, done, _ = env.step(action)
        pose = np.array([state[3], state[5], state[7]])
        if np.linalg.norm(env.desired_pose - pose) < 0.01:
            done = 1
        else:
            done = 0
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # if continuous action space; then decay action std of ouput action distribution
        if has_continuous_action_space and time_step % action_std_decay_freq == 0:
            ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)


            log_running_reward = 0
            log_running_episodes = 0

        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))

            print_running_reward = 0
            print_running_episodes = 0
            
        # save model weights
        if time_step % save_model_freq == 0:
            print("--------------------------------------------------------------------------------------------")
            print("saving model at : " + checkpoint_path)
            ppo_agent.save(checkpoint_path)
            print("model saved")
            print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
            print("--------------------------------------------------------------------------------------------")
            
        # break; if the episode is over
        if done:
            break

    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1

env.close()

# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")

training environment name : Dronesimscape
save checkpoint path : agent/PPO_preTrained/Dronesimscape/PPO_Dronesimscape_0_0.pth
--------------------------------------------------------------------------------------------
max training timesteps :  100000000
max timesteps per episode :  1000
model saving frequency : 20000 timesteps
log frequency : 2000 timesteps
printing average reward over episodes in last : 4000 timesteps
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Initializing a continuous action space policy
--------------------------------------------------------------------------------------------
starting std of action distribution :  None
--------------------------------------------------------------------------------------------
PPO update frequency : 4000 timesteps
PPO K epochs :  40
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
-

INFO: recv_socket: Socket not connected, nothing to close
INFO: send_socket: Socket not connected, nothing to close


Episode : 3 		 Timestep : 4000 		 Average Reward : -5172.48
Episode : 7 		 Timestep : 8000 		 Average Reward : -4090.63
Episode : 11 		 Timestep : 12000 		 Average Reward : -4078.38
Episode : 15 		 Timestep : 16000 		 Average Reward : -4377.57
Episode : 19 		 Timestep : 20000 		 Average Reward : -4353.27
--------------------------------------------------------------------------------------------
saving model at : agent/PPO_preTrained/Dronesimscape/PPO_Dronesimscape_0_0.pth
model saved
Elapsed Time  :  0:07:37
--------------------------------------------------------------------------------------------
Episode : 23 		 Timestep : 24000 		 Average Reward : -4117.56
Episode : 27 		 Timestep : 28000 		 Average Reward : -4049.31
Episode : 31 		 Timestep : 32000 		 Average Reward : -4101.23
Episode : 35 		 Timestep : 36000 		 Average Reward : -3827.77
Episode : 39 		 Timestep : 40000 		 Average Reward : -3064.91
----------------------------------------------------------------------------------

In [ ]:
# Test agent:
trajectory = []
sim_time = []
actions = []
done = False
while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    trajectory.append(state)
    sim_time.append(info["simulation time [s]"])
    actions.append(action)


In [ ]:
len(actions)

In [ ]:
# Stop the current episode:
env.stop_simulation()


In [ ]:
# Close environment:
env.close()
